In [296]:
import random
import functools
from ecpy.curves     import Curve,Point

cv = Curve.get_curve('secp256k1')
order = cv.order
P  = Point(0x65d5b8bf9ab1801c9f168d4815994ad35f1dcb6ae6c7a1a303966b677b813b00,
           0xe6b865e529b8ecbf71cf966e900477d49ced5846d7662dd2dd11ccd55c0aff7f,
           cv)
k  = 0xfb26a4e75eec75544c0f44e937dcf5ee6355c7176600b9688c667e5c283b43c5
Q  = k*P
R  = P+Q

n = 128
#for huge instance of n about 2048? and beyond the optimization seem to make sense
b= 256 # number of bits for scalar
c = 8 #c_limb_size or bit_size
parition_num = b//c
# scalars =[random.randint(0,b-1) for i in range(n)]
scalars =[random.randint(0,order-1) for i in range(n)]
points = [random.randint(0,b-1) * R for i in range(n)]
# print(compare)

In [297]:
compare = functools.reduce(lambda a,b: a+b,[scalars[i]*points[i] for i in range(n)],R+R.neg())
print("compare:", compare)
# About n mult and n addition

compare: (0xc516d77e06239801c77434998a13aae32b7d6f0473f8be4006b82ab019b2597a , 0x2435942a99483cdbbca10d62b495cd10676c90ee4350015a057423ab9ee66b9e)


In [298]:
# Naive decomposition

#Take from the lowest bit until the highest.
def take_bit(some_int,step,num_i):
    bin_repr = bin(some_int)[2:]
    bin_repr =  ((b - len(bin_repr)) * "0") + bin_repr 
    # bin_repr = bin_repr[::-1] # bin[0] is the lowest bit 1, follow by 2, 4, 8 etc.
    if (num_i ==0):
        return bin_repr[-step:]
    return bin_repr[-(step*(num_i+1)):(-(step*num_i))]

# T1 = first 3 bit of scalar1 * G1 + first 3 bit of scalar 2 * G2...
# T2 = next 3 bit of scalar1 *G2 + next 3 bit of scalar2 * G2 ...
# T_(b//c) = ...
part_T = []
for i in range(parition_num):
    acc = R+R.neg()
    for j in range(len(scalars)):  
        acc += (int(take_bit(scalars[j],c,i),base=2)*points[j])   
    part_T.append(acc)
part_T.reverse() # part_T the highest power is in front [0] for double-and-add

In [299]:
#Double and add with 2^c window.
def window_doubleandadd(part_T):
    T = part_T[0]
    for i in range(1,len(part_T)):
        T  = pow(2,c,order)*T
        T = T+ part_T[i]
    return T

print(window_doubleandadd(part_T))
print(compare)
assert window_doubleandadd(part_T) == compare
#about b//c multiplication and c addition
# for b==256, and c==8 [256 buckets] we have 32 mult and 32 addition 
print(parition_num)

(0xc516d77e06239801c77434998a13aae32b7d6f0473f8be4006b82ab019b2597a , 0x2435942a99483cdbbca10d62b495cd10676c90ee4350015a057423ab9ee66b9e)
(0xc516d77e06239801c77434998a13aae32b7d6f0473f8be4006b82ab019b2597a , 0x2435942a99483cdbbca10d62b495cd10676c90ee4350015a057423ab9ee66b9e)
32


In [300]:
part_T = []
addition_count =0
for i in range(parition_num):
    bucket_size =((2**c)-1)
    bucket_for_scalar = [R+R.neg() for _ in range(bucket_size)]
    for j in range(len(scalars)):  
        bit = int(take_bit(scalars[j],c,i),base=2)
        if (bit)!= 0:
            bucket_for_scalar[bit-1] += (points[j])
            addition_count+=1
    next_bucket = bucket_size-1
    rolling_sum = bucket_for_scalar[next_bucket]
    acc = rolling_sum
    next_bucket -= 1
    while next_bucket != -1:
        rolling_sum += bucket_for_scalar[next_bucket]
        acc += rolling_sum
        addition_count+=2
        next_bucket -= 1
    part_T.append(acc)
# two additions per bucket => 2**8 = 256*2 = 1024 additions 
print(addition_count)
part_T.reverse() # part_T the highest power is in front [0] for double-and-add

57030


In [301]:
print(window_doubleandadd(part_T))
print(compare)
assert window_doubleandadd(part_T) == compare

(0xc516d77e06239801c77434998a13aae32b7d6f0473f8be4006b82ab019b2597a , 0x2435942a99483cdbbca10d62b495cd10676c90ee4350015a057423ab9ee66b9e)
(0xc516d77e06239801c77434998a13aae32b7d6f0473f8be4006b82ab019b2597a , 0x2435942a99483cdbbca10d62b495cd10676c90ee4350015a057423ab9ee66b9e)
